In [489]:
import pandas as pd
import os
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import pymp
import time
import qgrid
from sklearn.metrics import roc_auc_score
from multiprocessing import Manager, cpu_count
from projects.feed.rank.utils.doc_info.KV import KV

In [490]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [491]:
# 全量
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8'
# 8
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/19'
# 16
info_dir = '/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1'

# info_dir = f'/search/odin/yuwenmengke/pikachu/projects/feed/rank/src/v25_tw_mktest_models_0430/tuwen_0430_tw_withScoreemb_withUseractemb_withDevemb_withDistrib_withKw_noSrch/infos'

hour = 2020072722
info_path = f'{info_dir}/{hour}/valid.csv'
print(info_path)
df = pd.read_csv(info_path)
# df2 = pd.read_csv()
# df = df.rename({'duration':'dur'})
df['dur'] = df.duration
df['mark'] = 'tuwen'
# info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8'
# info_path = f'{info_dir}/{hour}/valid.csv'
# df2 = pd.read_csv(info_path)
# df2['dur'] = df.duration
# df2['mark'] = 'video'
# df = pd.concat([df, df2])
if 'product_data' in df.columns:
  df = df[df.product_data=='sgsapp']
df.head()

/search/odin/publicData/CloudS/mkyuwen/rank_online/infos/video/1/2020072722/valid.csv


,history,keyword,topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,...,click,weight,cold,product_data,pred,pred_click,pred_dur,hour,dur,mark
0,0,0,0,13,0,6,2559180311477739905,3116,2070150K0duDu2,0,...,0,1.0,0,sgsapp,0.04948099,0.26654547,0.15897787,2020072722,0,tuwen
1,0,0,0,17,0,1,-2341823755939603764,3116,sf_458710302503428096,0,...,0,1.0,0,sgsapp,0.01864164,0.08729222,0.31334478,2020072722,0,tuwen
2,0,0,0,11,0,2,-5228596092562520829,3039,open_doc_prod36217269,0,...,0,1.0,0,sgsapp,0.03033586,0.24616528,0.09161043,2020072722,0,tuwen
3,0,0,0,2,0,3,5390518738956519996,3039,2072630K3S4E6r,0,...,0,1.0,0,sgsapp,0.02902559,0.09955925,0.46203882,2020072722,0,tuwen
4,0,0,0,7,0,2,1721616710454333027,8111,20724a0K1NmgJl,0,...,0,1.0,0,sgsapp,0.05788549,0.39215820,0.09710526,2020072722,0,tuwen


In [492]:
df.user_active

0          2
1          2
2          2
3          2
4          2
          ..
2517938    2
2517989   -1
2517990    0
2518041   -1
2518092    2
Name: user_active, Length: 1190566, dtype: int64

In [493]:
df.hour

0          2020072722
1          2020072722
2          2020072722
3          2020072722
4          2020072722
              ...    
2517938    2020072722
2517989    2020072722
2517990    2020072722
2518041    2020072722
2518092    2020072722
Name: hour, Length: 1190566, dtype: int64

In [494]:
df['click'] = (df.dur != 0).astype(int)

In [495]:
kv = KV()
titles = kv.get_titles(list(set(df.docid.values)))
df['title'] = df.docid.apply(lambda x: titles.get(x, ''))

get_titles: 100%|##########| 1112/1112 [00:46<00:00, 23.69it/s]


In [496]:
df['mark'] = df.title.apply(lambda x: '苟晶' in x)

In [497]:
df['click'] = (df.duration != 0).astype(int)

In [522]:
len(df[df.dur != 0]) / len(df)

0.14967502851584877

In [498]:
len(df[df.mark==True])

1

In [499]:
len(df[(df.mark==True)&(df.duration!=0)])

0

In [500]:
len(df[(df.mark==True)&(df.duration!=0)]) / len(df[df.mark==True])

0.0

In [501]:
df_doc = df.groupby('docid', as_index=False)["click"].agg({"shows" : "count", "clicks": sum})

In [502]:
df_doc['ctr'] = df_doc.clicks / df_doc.shows

In [503]:
df_doc

,docid,shows,clicks,ctr
0,09fc3dd81d66887a8df94bea3bbddf7c,7,0,0.00000000
1,15f3e2b88b172b37446ebeca5f721666,3,1,0.33333333
2,197170044C2nuf,1,0,0.00000000
3,19727m044afrjv,2,0,0.00000000
4,1980220443tQfd,3,1,0.33333333
...,...,...,...,...
111120,sv_206746138,1,0,0.00000000
111121,sv_206747319,13,3,0.23076923
111122,sv_206747584,6,0,0.00000000
111123,sv_206748718,2,0,0.00000000


In [504]:
df['dur2'] = df.dur.apply(lambda x: x if x > 0 else 0)
df['click2'] = df.dur.apply(lambda x: 1 if x > 0 else 0)

In [505]:
df_doc2 = df.groupby('docid', as_index=False)["dur2"].agg({"tdur" : sum})
df_doc3 = df.groupby('docid', as_index=False)["click2"].agg({"clicks2" : sum})

In [506]:
df_doc2

,docid,tdur
0,09fc3dd81d66887a8df94bea3bbddf7c,0
1,15f3e2b88b172b37446ebeca5f721666,202
2,197170044C2nuf,0
3,19727m044afrjv,0
4,1980220443tQfd,1
...,...,...
111120,sv_206746138,0
111121,sv_206747319,93
111122,sv_206747584,0
111123,sv_206748718,0


In [507]:
df_doc3

,docid,clicks2
0,09fc3dd81d66887a8df94bea3bbddf7c,0
1,15f3e2b88b172b37446ebeca5f721666,1
2,197170044C2nuf,0
3,19727m044afrjv,0
4,1980220443tQfd,1
...,...,...
111120,sv_206746138,0
111121,sv_206747319,3
111122,sv_206747584,0
111123,sv_206748718,0


In [508]:
df_doc = df_doc.merge(df_doc2, on='docid', how='left')
df_doc = df_doc.merge(df_doc3, on='docid', how='left')

In [509]:
df_doc

,docid,shows,clicks,ctr,tdur,clicks2
0,09fc3dd81d66887a8df94bea3bbddf7c,7,0,0.00000000,0,0
1,15f3e2b88b172b37446ebeca5f721666,3,1,0.33333333,202,1
2,197170044C2nuf,1,0,0.00000000,0,0
3,19727m044afrjv,2,0,0.00000000,0,0
4,1980220443tQfd,3,1,0.33333333,1,1
...,...,...,...,...,...,...
111120,sv_206746138,1,0,0.00000000,0,0
111121,sv_206747319,13,3,0.23076923,93,3
111122,sv_206747584,6,0,0.00000000,0,0
111123,sv_206748718,2,0,0.00000000,0,0


In [510]:
df_doc['mdur'] = df_doc.tdur / df_doc.clicks2
df_doc = df_doc.fillna(0)
df_doc.mdur = df_doc.mdur.astype(int)
df_doc

,docid,shows,clicks,ctr,tdur,clicks2,mdur
0,09fc3dd81d66887a8df94bea3bbddf7c,7,0,0.00000000,0,0,0
1,15f3e2b88b172b37446ebeca5f721666,3,1,0.33333333,202,1,202
2,197170044C2nuf,1,0,0.00000000,0,0,0
3,19727m044afrjv,2,0,0.00000000,0,0,0
4,1980220443tQfd,3,1,0.33333333,1,1,1
...,...,...,...,...,...,...,...
111120,sv_206746138,1,0,0.00000000,0,0,0
111121,sv_206747319,13,3,0.23076923,93,3,31
111122,sv_206747584,6,0,0.00000000,0,0,0
111123,sv_206748718,2,0,0.00000000,0,0,0


In [511]:
df = df.merge(df_doc, on='docid', how='left')

In [512]:
df

,history,keyword,topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,...,mark,title,dur2,click2,shows,clicks,ctr,tdur,clicks2,mdur
0,0,0,0,13,0,6,2559180311477739905,3116,2070150K0duDu2,0,...,False,白鹿原：兆鹏媳妇真可怜，被人欺负的走路都困难，还不敢言语啊!,0,0,5,1,0.20000000,141,1,141
1,0,0,0,17,0,1,-2341823755939603764,3116,sf_458710302503428096,0,...,False,20万工钱工期1月内的烟囱，4个农民工半天搞定，老板直接服了!,0,0,1763,129,0.07317073,23796,112,212
2,0,0,0,11,0,2,-5228596092562520829,3039,open_doc_prod36217269,0,...,False,菲律宾灵魂“炸鱼队”经典跳水，裁判都看懵了，一致给出0分!,0,0,9846,2522,0.25614463,130437,2306,56
3,0,0,0,2,0,3,5390518738956519996,3039,2072630K3S4E6r,0,...,False,盘点公交车上爆笑名场面，毛台放100多个屁，把车都崩翻了,0,0,1598,118,0.07384230,30568,98,311
4,0,0,0,7,0,2,1721616710454333027,8111,20724a0K1NmgJl,0,...,False,斗罗：比比东变身八腿蜘蛛精，蛛皇附体美妙绝伦，看完让人想叛变,0,0,1955,582,0.29769821,30767,542,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190561,0,0,0,8,0,1,1277768862065963497,3039,20706j041HCsvr,16,...,False,方一凡跳个舞成网红了，老师推荐他去艺考，这下可把文洁气坏了!,16,1,5,1,0.20000000,16,1,16
1190562,0,0,0,7,0,6,5806667384023407381,7160,207249044u6jO6,0,...,False,远嫁真的值得吗?看着马上出嫁的姐姐，心里五味杂陈,0,0,203,5,0.02463054,34,5,6
1190563,0,0,0,15,0,2,-8497908270362925852,3057,open_doc_prod36085731,0,...,False,计算9÷9÷9÷9，简单的题全班仅8人做对，老师提醒，按照运算顺序,0,0,252,23,0.09126984,2162,21,102
1190564,0,0,0,13,0,3,8246775750745475693,8542,20719m043ckEzt,0,...,False,范冰冰现在沦落成这样?跑到国外什么片都拍，这是什么都不顾了吗?,0,0,5472,2190,0.40021930,162514,1944,83


In [341]:
df_ = df.docid.value_counts().reset_index().head(20)
df_['title'] = df_['index'].apply(lambda x: titles.get(x, ''))
df_

,index,docid,title
0,open_doc_prod36229235,10402,500头野牛即将渴死，一辆水车出现之后，场面让人出乎意料
1,open_doc_prod36217269,9846,菲律宾灵魂“炸鱼队”经典跳水，裁判都看懵了，一致给出0分!
2,20716j0Q33vHVP,6861,吉林遍地俄罗斯美女，不工作靠什么谋生的?说出来你可能不信
3,open_doc_prod35993071,6831,大量游客挤上玻璃桥，桥面突然“碎裂”，镜头拍下全过程!
4,20719m043ckEzt,5472,范冰冰现在沦落成这样?跑到国外什么片都拍，这是什么都不顾了吗?
5,20713f0R4vqIS2,4234,看了无数次《极乐净土》，这妹子的版本腿最长
6,sf_471909839309078528,4056,大阅兵同样是在雨中阅兵，看看中国人民解放军!
7,207260042OOfhJ,3972,被课间操耽误的“舞王”，差点没把校长乐死，网友：是来砸场子的!
8,open_doc_prod36132919,3862,中国首辆飞行摩托来了，低空飞行10公里，还要啥汽车!只卖5万元
9,20726b0K1QwER3,3798,真脸假脸一吹便知：baby的脸，谢娜的脸，看到热巴：要保不住了?


In [369]:
def top_recall(df, abid):
  df_ = df[df.abtestid==abid]
  x = df_.docid.value_counts().head(20).reset_index()
  x = x.rename(columns={'index': 'docid', 'docid': 'count'})
  df_doc = df_.groupby('docid', as_index=False)["click"].agg({"shows" : "count", "clicks": sum})
  df_doc['ctr'] = df_doc.clicks / df_doc.shows
  x = x.merge(df_doc, on='docid', how='left')
  x['title'] = x['docid'].apply(lambda x: titles.get(x, ''))
  print(sum(x.clicks))
  print(sum(x.clicks) / sum(x.shows))
  return x

In [370]:
top_recall(df, 5)

1038
0.2350011319900385


,docid,count,shows,clicks,ctr,title
0,open_doc_prod36229235,570,570,194,0.34035088,500头野牛即将渴死，一辆水车出现之后，场面让人出乎意料
1,open_doc_prod36217269,485,485,126,0.25979381,菲律宾灵魂“炸鱼队”经典跳水，裁判都看懵了，一致给出0分!
2,open_doc_prod35993071,353,353,76,0.21529745,大量游客挤上玻璃桥，桥面突然“碎裂”，镜头拍下全过程!
3,20716j0Q33vHVP,348,348,129,0.37068966,吉林遍地俄罗斯美女，不工作靠什么谋生的?说出来你可能不信
4,20719m043ckEzt,255,255,111,0.43529412,范冰冰现在沦落成这样?跑到国外什么片都拍，这是什么都不顾了吗?
5,20713f0R4vqIS2,214,214,56,0.26168224,看了无数次《极乐净土》，这妹子的版本腿最长
6,207260042OOfhJ,213,213,36,0.16901408,被课间操耽误的“舞王”，差点没把校长乐死，网友：是来砸场子的!
7,open_doc_prod36132919,212,212,55,0.25943396,中国首辆飞行摩托来了，低空飞行10公里，还要啥汽车!只卖5万元
8,sf_471909839309078528,187,187,49,0.26203209,大阅兵同样是在雨中阅兵，看看中国人民解放军!
9,20726b0K1QwER3,178,178,33,0.18539326,真脸假脸一吹便知：baby的脸，谢娜的脸，看到热巴：要保不住了?


In [371]:
top_recall(df, 16)

935
0.21628498727735368


,docid,count,shows,clicks,ctr,title
0,open_doc_prod36229235,535,535,162,0.30280374,500头野牛即将渴死，一辆水车出现之后，场面让人出乎意料
1,open_doc_prod36217269,488,488,104,0.21311475,菲律宾灵魂“炸鱼队”经典跳水，裁判都看懵了，一致给出0分!
2,open_doc_prod35993071,353,353,78,0.22096317,大量游客挤上玻璃桥，桥面突然“碎裂”，镜头拍下全过程!
3,20716j0Q33vHVP,325,325,112,0.34461538,吉林遍地俄罗斯美女，不工作靠什么谋生的?说出来你可能不信
4,open_doc_prod36132919,250,250,51,0.20400000,中国首辆飞行摩托来了，低空飞行10公里，还要啥汽车!只卖5万元
5,20719m043ckEzt,249,249,102,0.40963855,范冰冰现在沦落成这样?跑到国外什么片都拍，这是什么都不顾了吗?
6,207260042OOfhJ,226,226,45,0.19911504,被课间操耽误的“舞王”，差点没把校长乐死，网友：是来砸场子的!
7,20713f0R4vqIS2,224,224,48,0.21428571,看了无数次《极乐净土》，这妹子的版本腿最长
8,sf_471909839309078528,202,202,35,0.17326733,大阅兵同样是在雨中阅兵，看看中国人民解放军!
9,20726b0K1QwER3,191,191,23,0.12041885,真脸假脸一吹便知：baby的脸，谢娜的脸，看到热巴：要保不住了?


In [372]:
top_recall(df, 17)

1082
0.23465625677727175


,docid,count,shows,clicks,ctr,title
0,open_doc_prod36217269,502,502,138,0.27490040,菲律宾灵魂“炸鱼队”经典跳水，裁判都看懵了，一致给出0分!
1,open_doc_prod36229235,468,468,144,0.30769231,500头野牛即将渴死，一辆水车出现之后，场面让人出乎意料
2,20716j0Q33vHVP,442,442,141,0.31900452,吉林遍地俄罗斯美女，不工作靠什么谋生的?说出来你可能不信
3,open_doc_prod35993071,423,423,97,0.22931442,大量游客挤上玻璃桥，桥面突然“碎裂”，镜头拍下全过程!
4,20719m043ckEzt,335,335,118,0.35223881,范冰冰现在沦落成这样?跑到国外什么片都拍，这是什么都不顾了吗?
5,20713f0R4vqIS2,249,249,64,0.25702811,看了无数次《极乐净土》，这妹子的版本腿最长
6,sf_471909839309078528,240,240,80,0.33333333,大阅兵同样是在雨中阅兵，看看中国人民解放军!
7,20614l0G4DGrp1,218,218,42,0.19266055,美国的未成年教育:警察局开设“监狱一日游”项目，家长把问题孩子送进监狱体验生活…
8,open_doc_prod27885140,187,187,16,0.08556150,解压爆浆球PK，小高改造无硼砂脆泥，一捏成型，为啥罚生吃洋葱
9,open_doc_prod24386598,169,169,15,0.08875740,整蛊无硼砂爆浆球大PK，竟然爆出100元，最后1个却是陷阱


In [346]:
x = df.docid.value_counts().head(20).reset_index()

In [347]:
x = x.rename(columns={'index': 'docid', 'docid': 'count'})

In [348]:
x = x.merge(df_doc, on='docid', how='left')
x['title'] = x['docid'].apply(lambda x: titles.get(x, ''))
x

,docid,count,shows,clicks,ctr,title
0,open_doc_prod36229235,10402,10402,3349,0.32195732,500头野牛即将渴死，一辆水车出现之后，场面让人出乎意料
1,open_doc_prod36217269,9846,9846,2522,0.25614463,菲律宾灵魂“炸鱼队”经典跳水，裁判都看懵了，一致给出0分!
2,20716j0Q33vHVP,6861,6861,2454,0.35767381,吉林遍地俄罗斯美女，不工作靠什么谋生的?说出来你可能不信
3,open_doc_prod35993071,6831,6831,1571,0.22998097,大量游客挤上玻璃桥，桥面突然“碎裂”，镜头拍下全过程!
4,20719m043ckEzt,5472,5472,2190,0.40021930,范冰冰现在沦落成这样?跑到国外什么片都拍，这是什么都不顾了吗?
5,20713f0R4vqIS2,4234,4234,1177,0.27798772,看了无数次《极乐净土》，这妹子的版本腿最长
6,sf_471909839309078528,4056,4056,1067,0.26306706,大阅兵同样是在雨中阅兵，看看中国人民解放军!
7,207260042OOfhJ,3972,3972,733,0.18454179,被课间操耽误的“舞王”，差点没把校长乐死，网友：是来砸场子的!
8,open_doc_prod36132919,3862,3862,915,0.23692387,中国首辆飞行摩托来了，低空飞行10公里，还要啥汽车!只卖5万元
9,20726b0K1QwER3,3798,3798,609,0.16034755,真脸假脸一吹便知：baby的脸，谢娜的脸，看到热巴：要保不住了?


In [349]:
pd.set_option('display.max_rows', 1000)
pd.set_option("display.precision", 8)

In [350]:
if 'article_page_time' not in df.columns:
  df.article_page_time = df.impression_time
df['page_time'] = (df.impression_time - df.article_page_time) / 60 / 60 / 24

In [351]:
df['impression'] = df.mid + '\t' + df.impression_time.astype(str)

In [352]:
mean_duration = df[df.dur > 0].dur.mean()

In [353]:
mean_duration

114.47596878330863

In [354]:
from projects.feed.rank.src import util
df['cold'] = df.rea.apply(lambda x: int(x) in util.cold_starts)
# df = df[df.product_data=='sgsapp']

In [355]:
import math
df.pred = df.pred.apply(lambda x: math.sqrt(x))

In [356]:
cold_mids = set(df[df.cold==True].mid.values)

In [357]:
len(set(df.mid.values))

116309

In [358]:
len(cold_mids)

2743

In [450]:
df[df.docid=='20717c0K2Ouv1h'].dur

58947        0
62898        3
101390       0
199050       0
252907       0
255019       0
272781       0
280777       0
285182       0
333751      -1
345185       0
498773       0
554944       0
595188       0
818190       0
843954       0
914131       0
963203       0
971730       0
1083477      0
1157564    176
1159033      0
Name: dur, dtype: int64

In [359]:
df.abtestid

0          13
1          17
2          11
3           2
4           7
           ..
1190561     8
1190562     7
1190563    15
1190564    13
1190565    19
Name: abtestid, Length: 1190566, dtype: int64

In [531]:
def debug(abid, mid=None, cold=False):
  while True:
    df_ = df[df.abtestid==abid]
    if mid is None:
      if not cold:
        mid = df_.mid.sample(1).values[0]
      else:
        mid = df_[df_.mid.isin(cold_mids)].mid.sample(1).values[0]
    cols = ['position', 'title', 'dur', 'ori_lr_score', 'pred', 'pred_click', 'pred_dur', 'shows', 'clicks', 'ctr', 'mdur', 'video_time', 'docid']
  #   if 'hour' in df.columns:
  #     cols += ['hour']
    d = df_[df_.mid==mid]
    print('mid:', mid, 'cold:', d.mid.values[0] in cold_mids, 'user_active:', d.user_active.values[0])
    start_time = d.impression_time.min()
    d['time'] = d.impression_time.apply(lambda x: int((x - start_time) / 60))
    cols += ['time']
    d = d.sort_values(['impression_time', 'position'])[cols].reset_index(drop=True)
#     d = d.sort_values(['impression_time', 'pred'], ascending=[True, False])[cols].reset_index(drop=True)
#     return d.sort_values(['ori_lr_score'], ascending=False)[cols].reset_index(drop=True)
    if d.dur.max() > 0:
      break
    else:
      mid = None
  return d

In [534]:
debug(5)

mid: b835861546034984716 cold: False user_active: -1


,position,title,dur,ori_lr_score,pred,pred_click,pred_dur,shows,clicks,ctr,mdur,video_time,docid,time
0,0,世界上最强壮的女人，被称为“女版施瓦辛格”，最大遗憾是没男票,0,2.0000000,0.00890624,0.16069546,0.03512016,1220,97,0.07950820,31,88,open_doc_prod35881401,0
1,2,奶奶做的脸盆一样大的凉粉 切开里面的结构 我瞬间没有了食欲。,0,1.6666666,0.00264308,0.13573042,0.00847313,1203,175,0.14546966,8,9,207194044GwHRT,0
2,6,一个只有女人和男孩的小岛，没有成年男性，传宗接代的任务只能靠男孩了,0,1.3333334,0.00767591,0.12534338,0.04505172,1892,198,0.10465116,86,176,20716h0R2mmZmo,0
3,0,我国最冷门的5个专业，录取分数线较低，毕业生却很吃香,0,2.0000000,0.00845084,0.20729494,0.02030581,1935,180,0.09302326,45,160,2072530K4d6uXT,4
4,3,范冰冰现在沦落成这样?跑到国外什么片都拍，这是什么都不顾了吗?,0,1.6666666,0.01207776,0.21073460,0.03280193,5472,2190,0.40021930,83,201,20719m043ckEzt,4
5,5,女人一旦没有了子宫，和正常人有什么不同?看完涨知识了,36,1.3333334,0.01253266,0.20889604,0.03514841,1888,302,0.15995763,54,77,20621g0Q1ihzbI,4


In [375]:
debug(8)

mid: b8d9867834048881563 cold: False


,position,title,mark,dur,ori_lr_score,pred,pred_click,pred_dur,docid,time
0,0,下一站是幸福：贺灿阳爆笑名场面：爸妈城市喂猪的愿望，终于实现,False,0,0.33345693,0.28453114,0.25246990,0.35526544,20614h0K0SwKzb,0
1,2,下一站：王耀庆大型追妹现场，我赌100包辣条，过程你肯定想不到,False,0,0.30482164,0.25031640,0.22525552,0.30448940,20720l0K0wbSSg,0
2,4,下一站：大学同学聚会带家属?宋茜被怼没男朋友，爆笑王耀庆救场,False,291,0.26564220,0.20230283,0.14923736,0.35593915,20626b0K4eMT17,0
3,6,下一站是幸福：有元宋的地方，贺灿阳永远都是那个背锅的人,False,0,0.26091440,0.20512309,0.16171446,0.31900215,20205l0431x7jF,0
4,8,速看《小欢喜》43：文洁督促俩孩子学习，身体不对劲，一查怀孕了,False,0,0.22864586,0.18021637,0.12901175,0.33526350,20717k0K3K3Gv2,0
5,10,庆余年：北齐大营遇刺，范闲却在旁坐山观虎斗，能不能有点同情心,False,0,0.19927444,0.16222423,0.15006160,0.18748769,20715k0K2R3ng3,0
6,0,下一站：刚追到女神，竟发现和总裁儿子亲密，王耀庆看见都哭了!,False,0,0.40091440,0.36662655,0.35545346,0.38831670,20708c0K1LdarQ,13
7,2,下一站：老丈人失踪王耀庆吓懵了，竟去湖里捞人，结局想哭又想笑,False,0,0.32993636,0.28391416,0.25573432,0.34474474,20709j0K0jugf5,13
8,4,下一站：心机男王耀庆追女神，挖墙脚首先搞定丈母娘，真是好手段,False,305,0.28652832,0.22487345,0.17313272,0.36545460,sf_465792538675470336,13
9,6,下一站是幸福12：元宋意外得知贺繁星疑似怀孕，认为孩子是他的,False,0,0.26099032,0.19341179,0.18119058,0.21833730,20707j0K1XR2I9,13


In [363]:
len(set(df.impression.values)) 

359437

In [364]:
len(set(df.mid.values))

116309

In [365]:
len(set(df.impression.values))  / len(set(df.mid.values))

3.0903627406305616

In [366]:
len(set(df[df.dur!=0].mid.values))

77967

In [367]:
len(set(df[df.dur!=0].mid.values)) / len(set(df.mid.values))

0.6703436535435778